In [1]:
import os
import json
import requests
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file
#OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
#os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
FMP_API_KEY: str = os.environ.get("FMP_API_KEY")

#FMP_API_KEY: str = os.environ.get("FMP_API_KEY")

client : OpenAI = OpenAI()


In [2]:
def get_income_statement(ticker, period, limit):
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_balance_sheet(ticker, period, limit):
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())


def get_cash_flow_statement(ticker, period, limit):
    # Code to fetch and return cash flow statement
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

    

def get_key_metrics(ticker, period, limit):
    url = f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

    # Code to fetch and return cash flow statement

def get_financial_ratios(ticker, period, limit):
    url = f"https://financialmodelingprep.com/api/v3/ratios-ttm/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

    # Code to fetch and return cash flow statement

def get_financial_growth(ticker, period, limit):
    # Code to fetch and return cash flow 
    url = f"https://financialmodelingprep.com/api/v3/financial-growth/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())


In [3]:
available_functions = {
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_cash_flow_statement": get_cash_flow_statement,
    "get_key_metrics": get_key_metrics,
    "get_financial_ratios": get_financial_ratios,
    "get_financial_growth": get_financial_growth
}


In [4]:
from openai.types.beta.threads import Run, ThreadMessage
from openai.types.beta.thread import Thread
from openai.types.beta.assistant_create_params import Tool

import time

class FinancialAssistantManager():
    def __init__(self,model:str = "gpt-3.5-turbo-1106"):
        self.client = OpenAI()
    
        self.model = model
        self.assistant = None
        self.thread = None
        self.run = None
    
    def create_assistant(self,name:str,instructions:str,tools:list[Tool]):
        self.assistant = self.client.beta.assistants.create(
            name = name,
            instructions=instructions,
            tools=tools,
            model=self.model
        )
    def create_thread(self) -> Thread:
        self.thread = self.client.beta.threads.create()
        return self.thread
    
    def add_messages_to_thread(self,role:str,content:str) -> ThreadMessage:
        self.client.beta.threads.messages.create(
            role=role,
            content=content,
            thread_id=self.thread.id
        )
    def run_assistant(self,instructions:str) ->Run: #changing this from run to run_assistant 
        self.run = self.client.beta.threads.runs.create( #yaha self lagaya hai 
            thread_id=self.thread.id,
            assistant_id=self.assistant.id,
            instructions=instructions    
        )
        return self.run
    
    def check_status(self,run : Run, thread : Thread) -> Run: #thread is not accessed -> THIS could be a problem. 
        while run.status in ["queued","in_progress"]:
            run_status = self.client.beta.threads.runs.retrieve(
                thread_id = self.thread.id,
                run_id = self.run.id
            )
            print(f'the status is {run_status} right now. ')
            time.sleep(5)

            if run.status == "completed":
                processed_message = self.process_message()
                return processed_message
            elif run.status == "requires_action":
                self.caller(run_status.required_action.submit_tool_outputs.tool_calls.model_dump())
            elif run.status == "failed":
                print("Run failed.")
                break
            else:
                print(f"Waiting for the Assistant to process...: {run.status}")

            

    def caller(self,actions:dict):
        tools_outputs = []
        for action in actions:
            function_name = action['function']['name']
            arguments = json.loads(action['function']['arguments'])
            print('function_name', function_name)
            print('function_arguments', arguments)
            if function_name in available_functions:
                function_to_be_called = available_functions[function_name]
                output = function_to_be_called(**arguments)
                tools_outputs.append({"output":output,"tool_call_id":action['id']})
            else:
                raise ValueError("unknown function, sorry.")
        
        print('submitting the values back to the assistant')
        self.client.beta.threads.runs.submit_tool_outputs(tool_outputs=tools_outputs,thread_id=self.thread.id,run_id=self.run.id)

    


    def process_message(self) ->list[ThreadMessage]:
        message: list[ThreadMessage] = self.client.beta.threads.messages.list(thread_id=self.thread.id)
        return message

                
            


In [5]:
financial_tools = [
    {"type": "code_interpreter"},
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string"},
                },
                "required": ["ticker"],
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string"},
                },
                "required": ["ticker"],
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_cash_flow_statement",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string"},
                },
                "required": ["ticker"],
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_key_metrics",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string"},
                },
                "required": ["ticker"],
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_financial_ratios",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string"},
                },
                "required": ["ticker"],
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_financial_growth",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string"},
                },
                "required": ["ticker"],
            }
        }
    }
]


In [6]:
INSTRUCTIONS = "Act as a financial analyst by accessing detailed financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends. "


In [7]:
def financialanalyst(prompt:str):
    fmp = FinancialAssistantManager()
    fmp.create_assistant(tools=financial_tools,name="assistant",instructions=INSTRUCTIONS) 
    fmp.create_thread()
    fmp.add_messages_to_thread(role="user",content=prompt)
    run = fmp.run_assistant(instructions=INSTRUCTIONS)
    finalres = fmp.check_status(run=run,thread=fmp.thread) #thora confused hain yaha
    return finalres


In [8]:

analysis1 = financialanalyst("Can you compare the financial health of Microsoft and Apple over the last four years, focusing on their balance sheets and key financial ratios?")
print(analysis1)

the status is Run(id='run_AJwuvtQ32LlLSn8AlAzsypiG', assistant_id='asst_6SFYxNjM3Ii7oMFx7rsChlAf', cancelled_at=None, completed_at=None, created_at=1702361802, expires_at=1702362402, failed_at=None, file_ids=[], instructions='Act as a financial analyst by accessing detailed financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends. ', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1702361803, status='in_progress', thread_id='thread_d3drWz8FFp5XBInfxCUb85Nn', tools=[ToolAssistantToolsCode(type='code_interpreter'), ToolAssistantToolsFunction(function=FunctionDefinition(name='get_income_statement', parameters={'type': 'object', 'properties': {'ticker': {'type': 'string'}}, 'required': ['ticker']}, description=None), type='function'), ToolAssistantToolsFunction(function=FunctionDefini

KeyboardInterrupt: 

In [ ]:
print('HEllo')